Implement conjoint analysis, A/B testing and RFM strategy

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import statsmodels.api as sm
import statsmodels.stats.api as sms
from statsmodels.stats.proportion import proportions_ztest

## Conjoint Analysis

In [2]:
df = pd.read_excel('conj_icecream.xlsx')

In [3]:
df.head()

,Observations,Flavor,Packaging,Light,Organic,Individual 1,Individual 2,Individual 3,Individual 4,Individual 5,Individual 6,Individual 7,Individual 8,Individual 9,Individual 10,Individual 11,Individual 12,Individual 13,Individual 14,Individual 15
0,Profile 1,Raspberry,Homemade waffle,No low fat,Not organic,1.0,6,5,1,2.0,7,7,5.0,1,10,1,10.0,7,10,5.0
1,Profile 2,Chocolate,Cone,No low fat,Organic,NaN,7,4,2,6.0,4,4,6.0,6,8,2,8.0,9,1,7.0
2,Profile 3,Raspberry,Pint,Low fat,Organic,2.0,1,6,5,1.0,8,8,2.0,2,9,6,9.0,4,9,3.0
3,Profile 4,Strawberry,Pint,No low fat,Organic,7.0,5,3,4,7.0,1,10,8.0,5,6,5,6.0,5,6,10.0
4,Profile 5,Strawberry,Cone,Low fat,Not organic,9.0,8,2,3,5.0,2,9,3.0,8,2,7,3.0,1,5,9.0


In this format we can't do conjoint analysis as our dependent variable (rank) is spread out. We need to transform the dataset so that instead of different columns of individuals we have different rows with corresponding ranking.

In [4]:
df1 = pd.melt(df, id_vars=df.columns.tolist()[:5], value_vars=df.columns.tolist()[5:])

In [5]:
# check
df1.head()

,Observations,Flavor,Packaging,Light,Organic,variable,value
0,Profile 1,Raspberry,Homemade waffle,No low fat,Not organic,Individual 1,1.0
1,Profile 2,Chocolate,Cone,No low fat,Organic,Individual 1,NaN
2,Profile 3,Raspberry,Pint,Low fat,Organic,Individual 1,2.0
3,Profile 4,Strawberry,Pint,No low fat,Organic,Individual 1,7.0
4,Profile 5,Strawberry,Cone,Low fat,Not organic,Individual 1,9.0


In [6]:
# rename columns for better information 
df1 = df1.rename(columns={'variable':'person','value':'rating'})

In [7]:
df1.head()

,Observations,Flavor,Packaging,Light,Organic,person,rating
0,Profile 1,Raspberry,Homemade waffle,No low fat,Not organic,Individual 1,1.0
1,Profile 2,Chocolate,Cone,No low fat,Organic,Individual 1,NaN
2,Profile 3,Raspberry,Pint,Low fat,Organic,Individual 1,2.0
3,Profile 4,Strawberry,Pint,No low fat,Organic,Individual 1,7.0
4,Profile 5,Strawberry,Cone,Low fat,Not organic,Individual 1,9.0


In [8]:
# check missing values
df1.isna().sum()

Observations    0
Flavor          0
Packaging       0
Light           0
Organic         0
person          0
rating          5
dtype: int64

In [9]:
# drop missing values as we can't use them in the regression
clean_df = df1.dropna()

In [10]:
# check if everything is correct
clean_df.isna().sum()

Observations    0
Flavor          0
Packaging       0
Light           0
Organic         0
person          0
rating          0
dtype: int64

In [11]:
# select dependent and independent variables
y = clean_df['rating']
x = clean_df[['Flavor', 'Packaging', 'Light', 'Organic']]
# make categories a dummy variables, as machines do not understand the text
xdum = pd.get_dummies(x)

In [12]:
# add the constant for a reference point
xdum = sm.add_constant(xdum)
# fit linear regression
res = sm.OLS(y,xdum).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 rating   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                 -0.031
Method:                 Least Squares   F-statistic:                    0.4514
Date:                Mon, 06 Feb 2023   Prob (F-statistic):              0.888
Time:                        17:39:05   Log-Likelihood:                -356.94
No. Observations:                 145   AIC:                             731.9
Df Residuals:                     136   BIC:                             758.7
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                         2.1480      0.104     20.582      0.000       1.942       2.354
Flavor_Chocolate             -0.2551      0.640     -0.398      0.691      -1.522       1.011
Flavor_Mango                  0.8763      0.538      1.629      0.106      -0.188       1.940
Flavor_Raspberry              0.1227      0.520      0.236      0.814      -0.905       1.150
Flavor_Strawberry             0.4380      0.554      0.790      0.431      -0.658       1.534
Flavor_Vanilla                0.9660      0.528      1.831      0.069      -0.077       2.009
Packaging_Cone                0.7853      0.565      1.389      0.167      -0.332       1.903
Packaging_Homemade waffle     0.6826      0.412      1.658      0.100      -0.132       1.497
Packaging_Pint                0.6801      0.382      1.778      0.078      -0.076       1.436
Light_Low fat                 0.9202      0.302      3.049      0.003       0.323       1.517
Light_No low fat              1.2277      0.297      4.129      0.000       0.640       1.816
Organic_Not organic           0.8727      0.249      3.505      0.001       0.380       1.365
Organic_Organic               1.2753      0.249      5.119      0.000       0.783       1.768
==============================================================================
Omnibus:                       38.565   Durbin-Watson:                   1.670
Prob(Omnibus):                  0.000   Jarque-Bera (JB):                7.562
Skew:                           0.037   Prob(JB):                       0.0228
Kurtosis:                       1.884   Cond. No.                     1.46e+16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.75e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [13]:
# create a new dataframe with attribute/level pairrs and their coefficient scores, which are the same as utility gains
df_res = pd.DataFrame({'param_name':res.params.keys(), 'param_w':res.params.values, 
                       'pval':res.pvalues}).reset_index(drop=True)

In [14]:
# separate attribute and levels
df_res['attr'] = df_res['param_name'].apply(lambda x: x.split('_')[0])
df_res['level'] = df_res['param_name'].apply(lambda x: x.split('_')[-1])

In [15]:
# go over unique attributes and calculate their utility range
attr_utils = {}
for i in list(df_res['attr'].unique())[1:]:
    a = df_res[df_res['attr']==i]
    attr_utils[i]=a['param_w'].max()-a['param_w'].min()

In [16]:
# make the dictionary a dataframe of attribute ranges
attr_utils_df = pd.DataFrame(attr_utils, index=[0]).T.rename(columns={0:'utils'})
# calculate the relative importance
attr_utils_df['rel_imp'] = attr_utils_df['utils'].apply(lambda x: (x/attr_utils_df['utils'].sum())*100)

In [17]:
attr_utils_df

,utils,rel_imp
Flavor,1.221079,59.964365
Packaging,0.105169,5.164627
Light,0.307490,15.100128
Organic,0.402602,19.770879


Based on the results we can see that the most important important attribute is Flavor with the Vanilla level.

The ideal product (bundle) will be composed of Vanilla flavor, Cone packaging, light no low fat, and organic

# A/B

In [18]:
ab = pd.read_csv('ab_data.csv')

In [19]:
ab.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [20]:
# assume each user should appear only once and check for the duplicates
# because we are not sure what records are right to keep we will remove all duplicated user_ids
ab[ab.duplicated(subset='user_id')]

,user_id,timestamp,group,landing_page,converted
2656,698120,2017-01-15 17:13:42.602796,control,old_page,0
2893,773192,2017-01-14 02:55:59.590927,treatment,new_page,0
7500,899953,2017-01-07 03:06:54.068237,control,new_page,0
8036,790934,2017-01-19 08:32:20.329057,treatment,new_page,0
10218,633793,2017-01-17 00:16:00.746561,treatment,old_page,0
...,...,...,...,...,...
294308,905197,2017-01-03 06:56:47.488231,treatment,new_page,0
294309,787083,2017-01-17 00:15:20.950723,control,old_page,0
294328,641570,2017-01-09 21:59:27.695711,control,old_page,0
294331,689637,2017-01-13 11:34:28.339532,control,new_page,0


In [21]:
# remove all duplicated user-ids
ab = ab[~(ab['user_id'].isin(list(ab[ab.duplicated(subset='user_id')]['user_id'].unique())))]

In [22]:
# group should have the same number of participants, check group number
ab['group'].value_counts()

treatment    143397
control      143293
Name: group, dtype: int64

In [23]:
# separate control and treatment groups
ab_control = ab[ab['group']=='control']
ab_treatment = ab[ab['group']=='treatment']

In [24]:
# randomly split the same proportion from tratment group equal to the number of control
ab_treatment= ab_treatment.sample(len(ab_control),random_state=42)

In [25]:
# create list of successes for each group- how many people converted out of the whole group
successes = [ab_control['converted'].sum(), ab_treatment['converted'].sum()]
# create count of observations
nobs = [len(ab_control), len(ab_treatment)]

In [26]:
zstat, pvalue = proportions_ztest(successes, nobs)

In [27]:
print('zstat: ', zstat)
print('pvalue: ', pvalue)

zstat:  1.2095853358972746
pvalue:  0.22643804784534916


Since our p-value=0.23 is higher than our α=0.05 threshold, we cannot reject the Null hypothesis Hₒ, which means that our new design did not perform significantly different (let alone better) than our old one.

# RFM

In [28]:
df = pd.read_excel('rfm_Online_Retail.xlsx')

In [29]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [30]:
# check for missing values
df.isna().sum()

InvoiceNo          0
StockCode          0
Description      166
Quantity           0
InvoiceDate        0
UnitPrice          0
CustomerID     25317
Country            0
dtype: int64

In [31]:
# drop missing values for CustomerID as we need to have that information for grouping and segmenting
df = df.dropna(subset=['CustomerID'])

In [32]:
# check for the duplicates
df.duplicated().sum()

633

In [33]:
# remove duplicated values
df = df.drop_duplicates()

In [34]:
# quantity and price should be positive check if we have negative values
df[df['Quantity']<0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
...,...,...,...,...,...,...,...,...
65097,C541693,22636,CHILDS BREAKFAST SET CIRCUS PARADE,-1,2011-01-20 17:02:00,8.50,14309.0,United Kingdom
65098,C541693,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,-6,2011-01-20 17:02:00,0.85,14309.0,United Kingdom
65099,C541694,22440,BALLOON WATER BOMB PACK OF 35,-10,2011-01-20 17:06:00,0.42,17364.0,United Kingdom
65100,C541694,22437,SET OF 9 BLACK SKULL BALLOONS,-10,2011-01-20 17:06:00,0.85,17364.0,United Kingdom


In [35]:
# assume that negative values is because of the data input and theya are correct values with the negative sign
df['Quantity'] = abs(df['Quantity'])
df['UnitPrice'] = abs(df['UnitPrice'])

In [36]:
#select only the needed columns
df1 = df[['Quantity','InvoiceDate','UnitPrice','CustomerID','InvoiceNo']]

In [37]:
# create total price column 
df1['total_price'] = df1['Quantity']*df1['UnitPrice']

/var/folders/wf/8rx2272d2zd2j3xnkrqf5sf80000gn/T/ipykernel_23630/4003477049.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['total_price'] = df1['Quantity']*df1['UnitPrice']


In [ ]:
# create rfm values 
rfm = df1.groupby(by='CustomerID').agg({'InvoiceDate': lambda date:(df1['InvoiceDate'].max()- date.max()).days,
                                 'InvoiceNo': 'count', 'total_price':'sum'})
rfm.columns = ['recency', 'frequency', 'monetaryvalue']

In [ ]:
# create quantile values such as 1 will be the best option
rfm['r_q'] = pd.qcut(rfm['recency'],4,['1','2','3','4']) # recent purchases are good
rfm['r_f'] = pd.qcut(rfm['frequency'],4,['4','3','2','1'])# high number of purchases
rfm['r_m'] = pd.qcut(rfm['monetaryvalue'],4,['4','3','2','1']) # high number of monetary values

In [ ]:
# join the quantiles and create the score 
rfm['RFM_score'] = rfm['r_q'].astype(str)+rfm['r_f'].astype(str)+rfm['r_m'].astype(str)

Afterwards you can filter based on the score and descirbe the groups